# Load Data

We're going to load test data to run into embeddings

## Install Prerequisites
First off, you'll need to install the Neo4j Graph Data Science Package. Please, check that you're running un a Python Kernel (Python 3)

In [1]:
%pip install --quiet --upgrade graphdatascience


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Working with Neo4j
You'll need to enter the credentials from your Neo4j instance below.  You can get these by running the command ":server connect" in the Neo4j Browser.  The default DB_USER and DB_NAME are always neo4j.

In [2]:
# Edit these variables!
DB_URL = "neo4j+s://f1691d5a.databases.neo4j.io"
DB_PASS = "g-OoYRW0zcFkUb9YncZu4C-2ZQEdXQI9QoAnGZcef-4"

# You can leave these defaults
DB_USER = "neo4j"
DB_NAME = "neo4j"

In [3]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS), aura_ds=True)

/Users/juancarlosrositocuellar/.pyenv/versions/3.7.9/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT IF NOT EXISTS FOR (p:Company) REQUIRE (p.cusip) IS NODE KEY
  """
)
display(result)

""


In [6]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT IF NOT EXISTS FOR (p:Manager) REQUIRE (p.filingManager) IS NODE KEY
  """
)
display(result)
    

""


In [8]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT IF NOT EXISTS FOR (p:Holding) REQUIRE (p.filingManager, p.cusip, p.reportCalendarOrQuarter) IS NODE KEY;
  """
)
display(result)
    

""


In [9]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
    MERGE (c:Company {cusip:row.cusip})
    SET
        c.nameOfIssuer=row.nameOfIssuer
  """
)
display(result)
    

""


In [11]:

result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
    MERGE (m:Manager {filingManager:row.filingManager})
  """
)
display(result)

""


In [12]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
    MERGE (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
    ON CREATE SET
        h.value=row.value, 
        h.shares=row.shares,
        h.target=row.target,
        h.nameOfIssuer=row.nameOfIssuer
  """
)
display(result)

""


In [13]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
    MATCH (m:Manager {filingManager:row.filingManager})
    MATCH (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
    MERGE (m)-[r:OWNS]->(h)
  """
)
display(result)

""


In [14]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
    MATCH (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
    MATCH (c:Company {cusip:row.cusip})
    MERGE (h)-[r:PARTOF]->(c)
  """
)
display(result)

""
